In [1]:
import torch
from torch import nn
from torchvision import datasets,transforms
from torch.utils.data import DataLoader,random_split
from torch import optim
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

In [2]:
transform=transforms.Compose([
    transforms.ToTensor(),#tranforms to the tensor
    transforms.Resize((256,256),),
    transforms.Normalize((0.5,),(0.5,))#just copied
])
root=r"C:\Users\saipr\Downloads\archive (1)prajith\Fruit And Vegetable Diseases Dataset"
dataset=datasets.ImageFolder(root=root,transform=transform)#it will divide classes w.r.t to no.of folders inside it

In [3]:
train_size=int(0.6*(len(dataset)))
val_size=int(0.2*(len(dataset)))
test_size=len(dataset)-train_size-val_size
train_data,val_data,test_data=random_split(dataset,[train_size,val_size,test_size])
train_loader=DataLoader(train_data,batch_size=64,shuffle=True)#train_data(that we are passing) should be tensor(my understanding)
val_loader=DataLoader(val_data,batch_size=64,shuffle=True)
test_loader=DataLoader(test_data,batch_size=64,shuffle=True)


In [4]:
class_names=dataset.classes#gives the  classes names
class_idx=dataset.class_to_idx

In [5]:
class CNN(nn.Module):
    def __init__(self,input_size,output_size):
        super().__init__()
        self.network=nn.Sequential(#intially I wrote self as nn
            nn.Conv2d(in_channels=3,out_channels=4,kernel_size=3,stride=1,padding=1),#these in channels are rbg colors of a colored photo
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(in_channels=4,out_channels=8,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(in_channels=8,out_channels=16,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Flatten(),     
            nn.Linear(32*8*8,64),
            nn.ReLU(),
            nn.Linear(64,output_size)            
        )
    def forward(self,x):
        return self.network(x)

In [6]:
model=CNN(input_size=256*256,output_size=len(dataset.classes))
criterion=nn.CrossEntropyLoss()
optimiser=optim.Adam(model.parameters(),lr=0.001)

In [7]:
num_epochs=20
train_losses,val_losses=[],[]
model.train()
for epoch in range(num_epochs):
 for input,output in train_loader:
    pred_output=model(input)
    loss=criterion(pred_output,output)
    
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()
 train_losses.append(loss.item()/len(train_loader))
 print(f'Loss in Epoch [{epoch+1}/{num_epochs}]:{loss} ')



C:\Users\saipr\AppData\Roaming\Python\Python312\site-packages\PIL\Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Loss in Epoch [1/20]:2.1163434982299805 
Loss in Epoch [2/20]:1.780981421470642 
Loss in Epoch [3/20]:1.4977457523345947 
Loss in Epoch [4/20]:1.1073744297027588 


In [ ]:

model.eval()
with torch.no_grad():
    for epoch in range(20):
        for input,output in val_loader:
            pred_out=model(input)
            loss=criterion(pred_out,output)
        val_losses.append(loss.item()/len(val_loader))

In [ ]:
def ClassificationReport(model,loader):
    y_test,y_pred=[],[]
    model.eval()
    with torch.no_grad():
        for input,output in loader:
            pred_output=model(input)
            y_test.extend(output.numpy())
            y_pred.extend(torch.argmax(pred_output,dim=1).numpy())
    model.train()
    print(classification_report(y_test,y_pred))
    
    for i in range(3):
        tc=sum(1 for y,x in zip(y_pred,y_test) if (y==i)&(x==i))
        ac=sum(1 for y in y_test if y==i)
        print(f'{list(class_idx.keys())[i]}: [{tc}/{ac}]')


In [ ]:
print('classification report of training phase:')
ClassificationReport(model,train_loader)


In [ ]:
print('Classification Report of validation set:')
ClassificationReport(model,val_loader)
plt.plot(train_losses,label='Training loss')
plt.plot(val_losses,label='validation loss')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('loss')
plt.show()

In [ ]:
print('Classification Report of testing phase:')
ClassificationReport(model,test_loader)